In [31]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

load_dotenv('.venv/.env')

API_KEY = os.getenv('OPENAQ_API_KEY')

BASE_URL = 'https://api.openaq.org/v3'
HEADERS = {
    'X-API-Key': API_KEY,
    'Accept': "application/json"
}

In [32]:
def get_measurements(sensor_id):
    url = f"{BASE_URL}/sensors/{sensor_id}/measurements/daily"
    response = requests.get(url, headers=HEADERS)
    return response.json()

def get_countries():
    url = f"{BASE_URL}/countries"
    response = requests.get(url, headers=HEADERS)
    return response.json()

def get_locations():
    url = f"{BASE_URL}/locations"
    response = requests.get(url, headers=HEADERS)
    return response.json()

locations = get_locations()
countries = get_countries()

In [33]:
sensor_list_by_countries = []
def filter_sensors_location():
    for item in locations.get('results', []):
        country_name = item.get('country', {}).get('name')
        for sensor in item.get('sensors', []):
            sensor_list_by_countries.append({
                'country': country_name,
                'sensor_id': sensor.get('id'),
                'sensor_name': sensor.get('name')
            })
    return sensor_list_by_countries

sensor_list_by_countries = filter_sensors_location()
df_sensors_by_countries = pd.DataFrame(sensor_list_by_countries)

In [34]:
print(df_sensors_by_countries)

Empty DataFrame
Columns: []
Index: []


In [ ]:
path = '/Users/jonatasgama/Desktop/air_quality_pipeline/data/raw/sensors_by_countries.csv'
df_path = pd.read_csv(path)
print(df_path.columns.tolist())

In [ ]:
values_by_sensor = []
def filter_values_by_sensor():
    path = '/Users/jonatasgama/Desktop/air_quality_pipeline/data/raw/sensors_by_countries.csv'
    df_path = pd.read_csv(path)
    for sensor_id in df_path['sensor_id']:
        measurements = get_measurements(sensor_id)
        for measurement in measurements.get('results', []):
            value = measurement.get('value')
            parameter_id = measurement.get('parameter').get('id')
            values_by_sensor.append({
                'sensor_id': df_path['sensor_id'],
                'parameter_id': parameter_id,
                'value': value
            })

    return values_by_sensor
    
values_by_sensor = filter_values_by_sensor()
df_values_by_sensor = pd.DataFrame(values_by_sensor)

In [ ]:
df_values_by_sensor.head()